In [88]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams ["figure.figsize"] = 10,6
import seaborn as sns
from scipy import stats
import math
import datetime
import urllib
import zipfile
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px
import glob
ohe = OneHotEncoder
le = LabelEncoder()
from sklearn.compose import ColumnTransformer
le = LabelEncoder()

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression as LR

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import cohen_kappa_score

In [89]:
bike_station = pd.read_csv('stations.csv')


In [131]:
bike_station.tail(5)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,15th & P St NW,31214,17th & Corcoran St NW,W01459,Member
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,Convention Center / 7th & M St NW,31201,15th & P St NW,W01262,Member
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,Lamont & Mt Pleasant NW,31602,Park Rd & Holmead Pl NW,W00998,Member
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,21st & I St NW,31222,New York Ave & 15th St NW,W00042,Member
1226766,468,2011-12-31 23:55:56,2012-01-01 00:03:45,31221,18th & M St NW,31111,10th & U St NW,W01319,Member


In [91]:
#FIX THE FRAME

#remove some uneccesary columns, change Duration to minutes, change members to two categories, make the categories workable. 

In [92]:
bike_station2 = bike_station.drop(['Start station', 'End station', 'Bike number'], axis=1)
bike_station2

,Duration,Start date,End date,Start station number,End station number,Member type
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual
...,...,...,...,...,...,...
1226762,300,2011-12-31 23:41:19,2011-12-31 23:46:20,31201,31214,Member
1226763,387,2011-12-31 23:46:43,2011-12-31 23:53:10,31223,31201,Member
1226764,261,2011-12-31 23:47:27,2011-12-31 23:51:49,31107,31602,Member
1226765,2060,2011-12-31 23:55:12,2012-01-01 00:29:33,31205,31222,Member


In [132]:
#most used stations?

#31200 for both start end end

In [94]:
bike_station2.dtypes

Duration                 int64
Start date              object
End date                object
Start station number     int64
End station number       int64
Member type             object
dtype: object

In [95]:
bike_station2["Start date"] = bike_station2["Start date"].astype('datetime64[ns]')

In [96]:
bike_station2["End date"] = bike_station2["End date"].astype('datetime64[ns]')

In [97]:
bike_station2["Duration"] = bike_station2["Duration"].astype('int64')

In [98]:
bike_station2.dtypes

Duration                         int64
Start date              datetime64[ns]
End date                datetime64[ns]
Start station number             int64
End station number               int64
Member type                     object
dtype: object

In [99]:
#bike_station2["Start date"]

In [133]:
minute_frame = (bike_station2["Duration"]) / (60)
#lst = []
#for each in minute_frame:
#    lst.append(str(each).split('.')[0])

#final_minute = [int(i) for i in lst]

#final_minute, done shows Duration in minutes! need to change to Int

In [101]:
bike_station2['dura_min']=(bike_station2['Duration']/60).round(0)

In [102]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0


In [103]:
bike_station2.dtypes


Duration                         int64
Start date              datetime64[ns]
End date                datetime64[ns]
Start station number             int64
End station number               int64
Member type                     object
dura_min                       float64
dtype: object

In [105]:
#MEMBERS FIX

In [106]:
bike_station2.loc[bike_station2['Member type'] == "Casual", 'Member cat'] = "1"
bike_station2.loc[bike_station2['Member type'] == "Member", 'Member cat'] = "2"
bike_station2.loc[bike_station2['Member type'] == "Unknown", 'Member cat'] = "1"

In [107]:
bike_station2.head(5)

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [108]:
bike_station2["Member cat"].info

<bound method Series.info of 0          2
1          1
2          2
3          2
4          1
          ..
1226762    2
1226763    2
1226764    2
1226765    2
1226766    2
Name: Member cat, Length: 1226767, dtype: object>

In [109]:
#fix the hot encoding

print(ohe)

<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [111]:
#DAY FIX

In [116]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [119]:
#TESTING AND SUCH

In [120]:
bike_station2.head()

,Duration,Start date,End date,Start station number,End station number,Member type,dura_min,Member cat
0,3548,2011-01-01 00:01:29,2011-01-01 01:00:37,31620,31620,Member,59.0,2
1,346,2011-01-01 00:02:46,2011-01-01 00:08:32,31105,31101,Casual,6.0,1
2,562,2011-01-01 00:06:13,2011-01-01 00:15:36,31400,31104,Member,9.0,2
3,434,2011-01-01 00:09:21,2011-01-01 00:16:36,31111,31503,Member,7.0,2
4,233,2011-01-01 00:28:26,2011-01-01 00:32:19,31104,31106,Casual,4.0,1


In [122]:
#x_data = bike_station2[["dura_min", "Start station number"]]
#y_data = new_cat[["Member cat new"]]


In [123]:
#display(x_data.head())
#display(y_data.head())

,dura_min,Start station number
0,59.0,31620
1,6.0,31105
2,9.0,31400
3,7.0,31111
4,4.0,31104


,Member cat new
0,1
1,0
2,1
3,1
4,0


# Split Data 

In [124]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.2, shuffle=False)
x_train, x_test, y_train, y_test

(        dura_min  Start station number
 0           59.0                 31620
 1            6.0                 31105
 2            9.0                 31400
 3            7.0                 31111
 4            4.0                 31104
 ...          ...                   ...
 981408       6.0                 31102
 981409      24.0                 31603
 981410       3.0                 31613
 981411      86.0                 31008
 981412       6.0                 31607
 
 [981413 rows x 2 columns],
          dura_min  Start station number
 981413       11.0                 31603
 981414       34.0                 31225
 981415        8.0                 31107
 981416      230.0                 31602
 981417        9.0                 31201
 ...           ...                   ...
 1226762       5.0                 31201
 1226763       6.0                 31223
 1226764       4.0                 31107
 1226765      34.0                 31205
 1226766       8.0                 3122

In [125]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train)

# transform training data
X_train_norm = norm.transform(x_train)

# transform testing data
X_test_norm = norm.transform(x_test)



In [126]:
# data normalisation with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x_train)

# transform training data
X_train_norm = norm.transform(x_train)

# transform testing data
X_test_norm = norm.transform(x_test)



In [127]:
#bike_station.isnull().any()

In [128]:
reg = LR().fit(x_train, y_train)

In [129]:
y_predicted = reg.predict(x_test)
y_predicted

array([[0.83480612],
       [0.73907081],
       [0.80961755],
       ...,
       [0.82172517],
       [0.73768898],
       [0.81749396]])

In [130]:
reg.score(y_predicted, y_test)

/Users/monicavillarfuentes/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: X has 1 features, but LinearRegression is expecting 2 features as input.

In [ ]:
display(y_predicted)
display(y_test.head())


array([[0.83480612],
       [0.73907081],
       [0.80961755],
       ...,
       [0.82172517],
       [0.73768898],
       [0.81749396]])

,Member cat new
981413,1
981414,1
981415,1
981416,0
981417,1


In [ ]:
reg.r2_score(y_test, y_predicted)

AttributeError: 'LinearRegression' object has no attribute 'r2_score'

In [ ]:
reg.coef_

array([-5.25730215e-05,  4.36012701e-05,  5.50321730e-05])

In [ ]:
reg.intercept_

-2.2253248156715673

In [ ]:
reg.predict #(np.array([[3, 5]])) ex.

<bound method LinearModel.predict of LinearRegression()>

In [ ]:
y_predicted = reg.predict(x_test)
y_predicted

array([0.80398807, 0.80409321, 0.71787972, ..., 0.85638507, 0.7451669 ,
       0.8234522 ])

In [ ]:
#Supply and Demand analysis (based on bike stations).

#take the most used start station vs. the most used end station.